# Step9 Replay Buffer

# 9 DQNには、replay　bufferが必要です。

　DQNという機械学習の特徴というか、方法のど真ん中を理解して、replay　bufferを準備します。

## 9.1 Q学習

Deep Q Network　の　Q　は何でしょうか？

どんな状態stateのときに、どんな行動actionをとるべきか？　これを方策といいます。ある行動を取ったら、報酬rewardが生じて、次の状態になります。この時に潜在価値のようなものをQとするならば、state0の時のQ(state0)と、行動をとった後の状態state1の時のQ(state1)は下のような関係になります。

Q(state0) = reward + Q(state1)

このQを行動価値関数とよびます。

この関係は、actionの種類だけ存在します。actionが変われば、その結果も報酬も変わるのは当たり前ですね。

このように、行動により次の結果が変わることをマルコフ過程と言います。　とるべき行動で次なる状態が多種多様になるときに、どのようにQは決まっていくのでしょうか？

## 9.2 だいたいで考えてみる。

　先のQの定義は、あるactionをした場合の価値なので、Qは多くのactionの集合体であると考えることができます。すると、actionが行われるであろう確率が分かれば、期待できる未来の潜在価値も予測できそうです。

　ところで、厳密な確率などわかるのでしょうか？　現実社会でも、ダメだと分かっていることには手を出さないように、確率などを算出することは無意味に見えます。でも、いつも同じ方法を選んでばかりいては、違うもっといい方法を知ることもできません。ランダムにやり方を変えるべきことと、潜在価値の高い方法を選ぶことのバランスが必要です。

　この「だいたい正しい答え」を探るのが、潜在価値を推測するコツになります。大切な部分です。

だいたい正しい答え　＝　（　ランダムな方法　：　潜在価値の高い方法　）　比率を調整

比率を調整する部分は、2つに分けられます。行動を起こす部分での調整と、学習する過程での調整です。行動を起こす部分はランダムに選択することをモンテカルロ法とも呼んでいます。そして、比率をイプシロンepsilonで表します。学習する過程では、replay_bufferを使います。

## 9.3 replay buffer
buffer_sizeの大きさの容器dequeにデータを蓄積していきます。超えると古い順になくなります。そして、batch_sizeをランダムに取り出します。

In [1]:
import numpy as np
import random
from collections import deque

class ReplayBuffer:
  def __init__(self, buffer_size, batch_size):
    self.buffer = deque(maxlen=buffer_size)
    self.batch_size = batch_size

  def add(self, state, action, reward, next_state, done):
    data = (state, action, reward, next_state, done)
    self.buffer.append(data)

  def __len__(self):
    return len(self.buffer)

  def get_batch(self):
    data = random.sample(self.buffer, self.batch_size)

    state = np.stack([x[0] for x in data])
    action = np.array([x[1] for x in data])
    reward = np.array([x[2] for x in data])
    next_state = np.stack([x[3] for x in data])
    done = np.array([x[4] for x in data]).astype(np.int32)
    return state, action, reward, next_state, done


## 9.4 replay bufferの役割
ランダムな方法とと潜在価値の高い方法の比率を、どのように変えていくかです。最初は、ほとんどランダムに行動します。推測値に価値がないからです。

学習が進むと、潜在価値が出てくるので、潜在価値に従って行動選択する比率を増やします。価値の高そうな方向に深堀していくイメージです。

潜在価値の高い行動が増えると、その学習をした方が効率がよくなります。ランダムに動いたムダな行動結果を学習する意味はほとんどありません。そこで、ある程度のデータが蓄積されてオーバーフローすると自動的に古いものから捨てると便利です。

この学習が進んで、深堀した時のデータを有効に使う仕組みこそが、replay bufferの役割です。

## 9.5 replay bufferのようなものは、他にはあるの？
　画像データの学習などでは、画像データを取り出すときに、ブロックに分割する、k分割交差検証があります。dataloaderと呼ばれるランダムにバッチファイルを取り出す方法もあります。この２つを組み合わせるのが王道です。